<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item"><li><span><a href="#Логистическая-регрессия" data-toc-modified-id="Логистическая-регрессия-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Логистическая регрессия</a></span></li><li><span><a href="#Случайное-дерево" data-toc-modified-id="Случайное-дерево-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Случайное дерево</a></span></li><li><span><a href="#CatBoostClassifier" data-toc-modified-id="CatBoostClassifier-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>CatBoostClassifier</a></span></li><li><span><a href="#LGBM-Classifier" data-toc-modified-id="LGBM-Classifier-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>LGBM Classifier</a></span></li></ul></li><li><span><a href="#Тестирование-лучшей-модели" data-toc-modified-id="Тестирование-лучшей-модели-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Тестирование лучшей модели</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Выводы</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.


**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

In [1]:
import pandas as pd
import numpy as np
import re
from pymystem3 import Mystem
import nltk
from nltk.corpus import stopwords as nltk_stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import f1_score
from sklearn.utils import shuffle
import warnings
import spacy
from tqdm.notebook import tqdm

In [2]:
warnings.filterwarnings('ignore')

In [3]:
data = pd.read_csv('/datasets/toxic_comments.csv')

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


In [5]:
data.head(20)

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0
5,5,"""\n\nCongratulations from me as well, use the ...",0
6,6,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1
7,7,Your vandalism to the Matt Shirvington article...,0
8,8,Sorry if the word 'nonsense' was offensive to ...,0
9,9,alignment on this subject and which are contra...,0


In [6]:
data['toxic'].value_counts()

0    143106
1     16186
Name: toxic, dtype: int64

Видно, что классы не сбалансированы, один примерно в 9 раз больше другого. Попробуем применить взвешивание классов для уравновешивания.

## Обучение

In [7]:
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

def lemmatize_text(text):
    text = text.lower()
    txt = nlp(text)
    lemm_text = " ".join([token.lemma_ for token in txt])
    cleared_text = re.sub(r'[^a-zA-Z]', ' ', lemm_text) 
    return " ".join(cleared_text.split())


tqdm.pandas()
data['lemm_text'] = data['text'].progress_apply(lemmatize_text)

data = data.drop(['text'], axis=1)

  0%|          | 0/159292 [00:00<?, ?it/s]

In [8]:
target = data['toxic']
features = data.drop(['toxic'], axis=1)

features_train, features_test, target_train, target_test = train_test_split(features, 
                                                                              target, 
                                                                              test_size=0.1, 
                                                                              random_state=12345)

nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('english'))

count_tf_idf = TfidfVectorizer(stop_words=stopwords)

features_train = count_tf_idf.fit_transform(features_train['lemm_text'])
features_test = count_tf_idf.transform(features_test['lemm_text'])

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Для обучения будем использовать следующие модели:
- логистическая регрессия
- случайное дерево
- CatBoostClassifier
- LGBM Classifier

### Логистическая регрессия

In [9]:
%%time

model_reg = LogisticRegression(class_weight='balanced')

print(f'''Значение метрики F1 на обучающей выборке для LinearRegression''', 
      cross_val_score(model_reg, features_train, target_train, 
                         cv = 3, 
                         n_jobs = -1, 
                         scoring = 'f1').mean())

Значение метрики F1 на обучающей выборке для LinearRegression 0.748837762610259
CPU times: user 59.4 s, sys: 1min 10s, total: 2min 9s
Wall time: 2min 9s


### Случайное дерево

In [10]:
%%time

model_tree = DecisionTreeClassifier(random_state=12345)
max_depth_list = [x for x in range(2, 31)]
parameters_tree = {'max_depth': max_depth_list}
grid = GridSearchCV(model_tree, param_grid = parameters_tree, cv=3, scoring='f1')
grid.fit(features_train, target_train)
display(grid.best_params_)
display(grid.best_score_)

{'max_depth': 30}

0.6878381351729321

CPU times: user 20min 53s, sys: 1.45 s, total: 20min 54s
Wall time: 20min 54s


### CatBoostClassifier

In [11]:
%%time

model_cat = CatBoostClassifier(cat_features=[], verbose=False, random_state=12345)
parameters_cat = {'learning_rate':[0.1, 0.5, 0.8]}
grid = GridSearchCV(model_cat, parameters_cat, cv=3, scoring='f1')
grid.fit(features_train, target_train)
display(grid.best_params_)
display(grid.best_score_)

{'learning_rate': 0.5}

0.7651161842980562

CPU times: user 4h 38min 50s, sys: 2min 7s, total: 4h 40min 57s
Wall time: 4h 41min 13s


### LGBM Classifier

In [12]:
%%time

model_lgbm = LGBMClassifier(random_state=12345)
parameters_lgbm = {'num_leaves' : [100, 150, 200]}
grid = GridSearchCV(model_lgbm, parameters_lgbm, cv=3, scoring='f1')
grid.fit(features_train, target_train)
display(grid.best_params_)
display(grid.best_score_)

{'num_leaves': 200}

0.7758700637962693

CPU times: user 1h 15min 30s, sys: 10.8 s, total: 1h 15min 41s
Wall time: 1h 16min 31s


Таким образом, лучшая модель на кросс-валидации - LGBM Classifier. Протестируем ее на тестовой выборке.

## Тестирование лучшей модели

In [14]:
model_lgbm = LGBMClassifier(random_state=12345, num_leaves=200)
model_lgbm.fit(features_train, target_train)
pred = model_lgbm.predict(features_test)
f1_score(target_test, pred)

0.7909527073337903

Модель показывает достаточно высокую метрику F1.

## Выводы

В ходе работы были обработаны данные (в частности, применено взвешивание классов для броьбы с дисбалансом), обучено с кросс-валидацией несколько моделей, выбрана лучшая и протестирована на тестовой выборке, где показала F1-меру 0.79, что является достаточно высоким показателем для поставленной задачи.